<a href="https://colab.research.google.com/github/parthrjpt/EAS555_Projects/blob/main/parthket_cse555_finalreport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Reliability of New Articles: Identifying Real and Fake News**

##**Introduction**

News Articles play an important role in our daily lives. They keep us updated regarding the daily proceedings around the world. They play an important role. For example, financial articles such as changes in stock market prices, articles on topics like cryptocurrency, or the federal budget, impact the common man. These help the common man to control their budget and think about their investments. News also conveys this information to various stakeholders such as local vendors, which control the rates of goods and services, which in turn impacts our regular life. News is also a medium of spreading social awareness not just locally but on a global scale. It tells us about the daily events that take place worldwide. Due to such an impact of news on human lives, we can agree with Edmund Burke, when he stated that the Fourth Estate, that of news, and media is far more important. It is also due to this importance it is much more vulnerable. Due to the emergence of various social media platforms, it is becoming more and more difficult to authenticate the source of news, making it much easier to fake it. A spread of such information, not only leads to confusion and chaos but also can lead to catastrophic losses, 

This problem statement of classifying fake/unreliable news articles apart from real/reliable news can be resolved with the help of various machine learning algorithms. A simple implementation of a decision tree or a binary classifier can be used provided the dataset is noise-free and annotated. This project report and the corresponding project intend to use a Passive-Aggressive classifier to provide an optimal solution.


### **DataSet**
For this project, we will be utilizing the Fake News dataset which was published on Kaggle for an InClass Prediction Competition. It comprises 20800 training samples and around 5200 test samples provided in csv formats. There are 5 columns: id, title, author, test, label [1] : 

*   id: unique id for a news article
*   title: the title of a news article
*   author: author of the news article
*   text: the text of the article; could be incomplete
*   label: a label that marks the article as potentially unreliable 





In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle
!mkdir ~/.kaggle
!echo '{"username":"parthketanrajput","key":"fec6fb4996a62e180eda7488c75580c9"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

     |████████████████████████████████| 61kB 5.9MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=0d06b85e692b0b75e19878cad082c57a1af6c6edc4a91f0493ed6b0ee65a2b9b
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
!kaggle competitions download -c fake-news

 88% 41.0M/46.5M [00:00<00:00, 28.0MB/s]
100% 46.5M/46.5M [00:01<00:00, 48.2MB/s]


In [ ]:
import zipfile
with zipfile.ZipFile('./fake-news.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

##**Methods** 
The dataset is segregated into testing and training data.However, the test data is not annotated. The dataset was intended for competitive purposes to identify the optimal apporach for the problem statement. Therefore, the training data will be split for training and validation datasets. The dataset consists of two test features (i.e. title and text) of which the text feature is the major contributor for determining whether the article in question is reliable or not. The text feature contains a lot of irrelevant data, which should not be considered. This is because this data contains free-text  (which is understandable by humans but not by a machine). To resolve this issue, relevant words/terms need to be identified and irrelevant stop words must be removed. This is done with the help of tf-idf (term frequency/ inverse document frequency). Tf-idf is a statistical measure that identifies the relevancy of a term concerning a document. The term frequency is the number of occurrences of a term in a particular document. The inverse document frequency is the measure of occurrences of a term across a set of documents. Tf-idf provides a vectorized value that can be further utilized as features.
For classification, a passive-aggressive approach is used. These approaches are usually implemented for large-scale learning [2]. Such algorithms are usually called ‘online learning’ algorithms, wherein the data is fed to the model sequentially instead of being fed as an entire batch. In these algorithms, if the prediction is correct then the model remains unchanged, else the model is updated.
 
The trained model wis then used to predict the labels (denoting the reliability of the news articles). The accuracy of the model is defined with the help of a confusion matrix and accuracyscore since to indicate the actual number of correct classifications. 

The Passive agressive approach is also implemented with the help of countvectorization and the performance of both models is compared. Logistic regression is also used to determine output variables for validation data.

Based on the comparitive performance of all three approaches, the approach with the highest accuracy is used for the predicting output variables for test data.


### **Passive Agressive Classifiers**





For this project, Passive Aggressive CLassifier (PA) has been implemented. It is a family of online learning algorithms which can be used for binary and multiclass classification regression, etc.[3] This problem statement implemnts the Passive Agressive Classifier as a binary categorizer, that determines if a news article is authentic or not.

Like every online algorithm, the PA Classifier follows a sequential approach. The algorithm will make a prediction like a binary classfier problem, it will receive a feedback and based on it, it may modify its prediction mechanism. 

The PA classifer, represents each instance as a vector and utilizes a decision boundary which modifies the state of the classifier.

The algorithm for a binary classification problem will follow the following steps:
*  Define an agressiveness paramter C>0
*  Initialize weight w1= (0,....,0) 
*  For each example i.e ann(xi,yi) pair:
    *  predict yt= sign(wt,xt)
    *  Receive correct label and compute hinge loss
    *  Define τ and update the weights for the next iteration

In [ ]:
#import dependancies
import numpy as np
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import feature_extraction,pipeline
import nltk
from nltk.corpus import stopwords


In [ ]:
#Load dataset
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')


In [ ]:
#Identify missing data values
missingdatacount = train_data.isnull().sum()

In [ ]:
missingdatacount

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
#filling empty datavalues
train_data = train_data.fillna('')

In [ ]:
test_data = test_data.fillna('')

In [ ]:
y=train_data.label

In [ ]:
#Splitting the dataset for trainng and validation purposes
X_train, X_val, y_train, y_val = train_test_split(train_data['text'].astype(str), y, test_size=0.2)

In [ ]:
#The text field is the only field relevant for this implmentation
X_test=test_data['text'].astype(str)

In [ ]:
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(16640,) (4160,) (16640,) (4160,)


In [ ]:
X_train.head()

17640    MONTAGUE, Mass.  —   Think of all the dogs out...
14615    In his speech at Washington University, MILO c...
2554     SEOUL, South Korea  —   It was supposed to be ...
16550    Seven World-Historical 'Achievements' Of The I...
3179     By Peter van Els, The Netherlands Many sometim...
Name: text, dtype: object

In [ ]:
# Download stopwords for english language
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
corpus=set(stopwords.words('english'))

In [ ]:
#Remove stopwords
data_train = [word for word in X_train if word not in (corpus)]
data_val =  [word for word in X_val if word not in (corpus)]
data_test= [word for word in X_test if word not in (corpus)]

In [ ]:
# Utilizing TF-IDF Vectorization to identify relevant term frequencies in documents and replace them with a numerical equivalent value
tfidf = feature_extraction.text.TfidfVectorizer()
tfidf_train = tfidf.fit_transform(data_train)
tfidf_val = tfidf.transform(data_val)
tfidf_test = tfidf.transform(data_test)

In [ ]:
clf=PassiveAggressiveClassifier(max_iter=50)

In [ ]:
clf.fit(tfidf_train,y_train)

PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
                            early_stopping=False, fit_intercept=True,
                            loss='hinge', max_iter=50, n_iter_no_change=5,
                            n_jobs=None, random_state=None, shuffle=True,
                            tol=0.001, validation_fraction=0.1, verbose=0,
                            warm_start=False)

In [ ]:
y_pred_val1 = clf.predict(tfidf_val)

In [ ]:
#Alternatively using Count Vectorizer.
count_vectorizer = feature_extraction.text.CountVectorizer()
cv_train = count_vectorizer.fit_transform(data_train)
cv_val = count_vectorizer.transform(data_val)

In [ ]:
clf.fit(cv_train,y_train)

PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
                            early_stopping=False, fit_intercept=True,
                            loss='hinge', max_iter=50, n_iter_no_change=5,
                            n_jobs=None, random_state=None, shuffle=True,
                            tol=0.001, validation_fraction=0.1, verbose=0,
                            warm_start=False)

In [ ]:
y_pred_val2 = clf.predict(cv_val)  

In [ ]:
from sklearn.linear_model import LogisticRegression

lclf= LogisticRegression(max_iter=100)
lclf.fit(tfidf_train,y_train)
y_pred_lgr=lclf.predict(tfidf_val)

clf= PassiveAggressiveClassifier(max_iter=50)
clf.fit(tfidf_train,y_train)
y_pred_test = clf.predict(tfidf_test)

##  **Results**

The above implementation demostrates how the dataset has been fit on the model. The Passive aggressive classifer was implemented with both the Count vectorizer and  TF-IDF vectorizer. Based on the below performance measures, it can be said that TF-IDF has a much higher performance on the validation dataset. The model is then compared in similar settings with Logisitic regression on the same model. In this case as well, the Passive Aggressive Classifier performs better comparitively.

Based on these results we utilize a similar approach we then classify the test dataset with a PA Model. Since this was dataset meant for competitive purposes, it lacks actual output variables for the test data and therefore the predicited results cannot be verified.


In [ ]:
confusion_matrix(y_val,y_pred_val1)

array([[2000,   83],
       [  76, 2001]])

In [ ]:
confusion_matrix(y_val,y_pred_val2)

array([[1939,  144],
       [ 150, 1927]])

In [ ]:
accuracy_score(y_val,y_pred_val1)

0.9617788461538461

In [ ]:
accuracy_score(y_val,y_pred_val2)

0.9293269230769231

In [ ]:
confusion_matrix(y_val,y_pred_lgr)

array([[1941,  142],
       [ 114, 1963]])

In [ ]:
accuracy_score(y_val,y_pred_lgr)

0.9384615384615385

In [ ]:
y_pred_test

array([0, 1, 1, ..., 0, 1, 0])

### **Citations**
 
[1] Fake News Dataset ( https://www.kaggle.com/c/fake-news/overview/evaluation)<br/>
[2] Passive Aggressive Classifiers (https://www.geeksforgeeks.org/passive-aggressive-classifiers/)</br>
[3] Koby Crammer, Ofer Dekel, Joseph Keshet, Shai Shalev-Shwartz, and Yoram Singer. 2006. Online Passive-Aggressive Algorithms. J. Mach. Learn. Res. 7 (12/1/2006), 551–585.(https://jmlr.csail.mit.edu/papers/volume7/crammer06a/crammer06a.pdf)

 


# **Ethics in AI**

## **Impact on the problem statement**
AI is a major aspect of applications that vary across various areas of exterise globally. Some of these applications are as trivial as performing a google search, whereas others are as critical as identifying tumours and categrozing as malignant or benign for a patient. Due to this, the ethicality of the implementations is a critical factor that must be accounted for.

Given that some of these applications, are critical and can lead to breach of trust, security hazards, financial losses, it is critical that standards and regulations be imposed and an awareness be raised among people.

Consider the above problem statement for fake news, as it is already a known fact that data is the most significant component when it comes to AI. If the data is biased, then the results are also bound to be biased. SOurces of all over the internet can ideally be categorized as genuine or not. However, consider a scenario, wherein a reputated news agency becomes interested in increasing its viewership and tries to fabircate news or exagerrate it. In such a case, the article will even be able to fool a human. If such articles are fed to the training model, then there are bound to be faulty classifications. Since, news articles are a critical part of human society, conveying fake/unreliable articles to the people can have disastrous outcomes.

In this scenario, a centralized governing body that authenticates all news sources and verifies them with other articles(major or minor) is necessary. If an inconsistency is observed, then the article must be investigated. Standard rules and regulations, and penalties can be imposed to ensure the autheticity and reliability of the data source.

While only one aspect of maintaining Ethics in AI has been discussed, other aspects such as impact on privacy of the people should also be consideder.

## **Impact on Employment**

As previously mentioned, AI has a significant reach in every sector. In sectors like public safety, manufacturing, etc. which were once cosnidered as job profiles that require physical efforts, AI along with robotics has managed to replace the human factor, achieving much more efficiency. However, this also meant that a lot of these job profiles became obsolete and increase in  unemployment. The IT sector is also no exception to this change. A lot of software development organizations, are heavily integrating Ai to improve user experience in their products. However, they now also use Ai based tools for automation testing and unit testing. Due to this the industry has a lower demand for Quality Analysts/Testers in such organizations. It can be said that over the next few years, the job profile may become compeletely obsolete.

While technological advancements are indeed a necessity, we also need to realise their indirect impact. In this case, an analysis of the implementation must be taken into consideration, and restrictions must be imposed on the extent to which job profiles can automated. Limitations to automation, can also be implemented, by raising awareness related to expenses, and ethicality of it. 


